In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# from imitation.data import rollout
import pickle


# import seals  # noqa: F401
import stable_baselines3 as sb3
from stable_baselines3 import PPO, SAC
from stable_baselines3.ppo import MlpPolicy

import gym
# from imitation.algorithms.bc import BC
from stable_baselines3.common.policies  import ActorCriticPolicy, ActorCriticCnnPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np

from stable_baselines3.common.env_util import make_vec_env 

/home/ns/anaconda3/lib/python3.9/site-packages/botocore/httpsession.py:34: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In [3]:
env_name='Pendulum-v1' 
env = gym.make(env_name)

In [4]:
savepath= "trained_models/Pendulum-v1.zip"
model=SAC.load(savepath, print_system_info=False)

In [5]:
def collect_trajectory(env, policy_net, n_trajectory=20, epsilon=0.0):
    trajectories=[] 
    scores=[]
    for episode in range(n_trajectory):
        state,info = env.reset()
        score = 0 
        states=[]
        actions=[]
        rewards=[]
        while True:
            if epsilon==0:
                action, _states = policy_net.predict(state, deterministic=True)
            else:
                if np.random.random() > (1-epsilon):
                    action=env.action_space.sample() 
                else:
                    action, _states = policy_net.predict(state, deterministic=True)
            
            next_state, reward, done,s, _ = env.step(action)
            score+=reward
             
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            state = next_state 
            if done or s:  
                scores.append(score)
                break
        
        states=np.vstack(states)
        actions=np.vstack(actions)
        rewards=np.vstack(rewards)
        trajectories.append((states, actions, rewards))  
        if episode % 1 == 0:
            print('{} episode score is {:.2f}'.format(episode, score))
    env.close()
    return scores, trajectories

In [6]:
print('collecting trajectories ...')
n_trajectory=50
scores, trajectories = collect_trajectory(env , model, n_trajectory)
mean=np.mean(scores)
print('mean score:', mean)

collecting trajectories ...
0 episode score is -251.53
1 episode score is -118.59
2 episode score is -132.65
3 episode score is -122.70
4 episode score is -121.26
5 episode score is -232.95
6 episode score is -123.39
7 episode score is -121.60
8 episode score is -118.17
9 episode score is -226.21
10 episode score is -125.43
11 episode score is -226.83
12 episode score is -226.39
13 episode score is -223.10
14 episode score is -227.38
15 episode score is -117.16
16 episode score is -117.08
17 episode score is -125.65
18 episode score is -224.37
19 episode score is -228.14
20 episode score is -120.54
21 episode score is -1.25
22 episode score is -125.47
23 episode score is -1.26
24 episode score is -116.74
25 episode score is -124.98
26 episode score is -116.22
27 episode score is -121.03
28 episode score is -1.97
29 episode score is -128.15
30 episode score is -227.68
31 episode score is -237.49
32 episode score is -121.13
33 episode score is -121.38
34 episode score is -123.70
35 episo

In [7]:
import pickle

filename=f"expert_data/{env_name}_{n_trajectory}_{int(mean)}.pkl"
print('saving ',filename)

with open(filename, 'wb') as f:
    pickle.dump(trajectories, f)
print('trajectories saved.')

saving  expert_data/Pendulum-v1_50_-149.pkl
trajectories saved.
